#### Note:
This is a Jupyter Notebook, which means all code sections are actually executed live. Please select "Cell -> Run All" before reading for the first time to generate the plots/widgets.

#### Note:
Some raw code for this IPython notebook is hidden by default for easier reading. If you want to see it, clicking on the toggle buttons will allow you to read and edit it.

In [3]:
from IPython.display import HTML as DHTML

# function to create those toggle buttons, adapted from something from stackoverflow
def add_toggle_button(desc, *inputs):
    x = ', '.join([str(x) for x in inputs])
    n = str(inputs[0])
    
    code = '''
        <script>
        var others%n = [%x];
        var code_shown%n = true; 
        function code_toggle%n () {
            var selector = "div.input";
            var inputs = $(selector).toArray();
            if (code_shown%n) {
                for (var i in others%n) {
                    var x = others%n[i];
                    $(inputs[x]).hide();
                }
            }
            else {
                for (var i in others%n) {
                    var x = others%n[i];
                    $(inputs[x]).show();
                }
            }

            code_shown%n = !code_shown%n;
        } 
        $( document ).ready(code_toggle%n);
        </script>
        <form action="javascript:code_toggle%n()">
        <input type="submit" value="Toggle on/off the display of the %d code.">
        </form>'''
    
    return code.replace('%d', desc).replace('%n', str(n)).replace('%x', x)

DHTML(add_toggle_button('document setup', 0, 1, 2))

In [4]:
%%html
<style>
/* prevent truncation of the slider labels */
.widget-label, .widget-button, .jupyter-button, .widget-readout {
    width: unset !important;
    min-width: fit-content !important;
}
.widget-vbox > .widget-label {
    font-weight: bolder;
}
</style>

In [5]:
import numpy
from bqplot import LinearScale, Axis, Lines, Figure, Hist, Bars, Scatter
from ipywidgets import HBox, VBox, SelectionSlider, Button, Label, Layout, ToggleButton, HTML
from ipywidgets.widgets.widget_description import DescriptionStyle

initial_harvester = 'Adaptive'

# baseline samples-per-harvest, which is set to "10" in
# the agent spec
baseline_sph = 10 

# harvests per hours
initial_hph = 60

# moving-average range
initial_ma_range = 1

tidy_output = True
initial_traffic_model = 'Varying'

initial_seed = 1234567
initial_erlang_window_size = 10
initial_num_bpf = 5
initial_rem = 'Mean'

# Investigation into Adaptive Sampling

This notebook contains some simulations I created to investigate the performance of the Better CAT v1 adaptive sampling algorithm. 

I gathered some data from a random customer on the number of transactions they were generating for a random 24 hour period. (well, the customer wasn't selected truly at random... it was just the APM link from the last support ticket I had worked on when I started this notebook.)

`t_per_hour` is the data I came up with:

In [6]:
# APM data for the Python/RequestSampler/requests metric,
# which describes the number of transactions seen in
# a given harvest. It is lumped into per-hour counts when
# inspected in APM. So t_per_hour[0] is from midnight to
# 1:00am, t_per_hour[1] is 1:00am to 2:00am, etc.

t_per_hour = [
    294, 254, 325, 516, 886, 3022,
    10109, 16507, 22262, 24752, 26362, 25034,
    27172, 27208, 26751, 20724, 14936, 10160,
    5808, 3053, 1397, 571, 152, 351
]

# Simulation

Next, I implemented a Mock Agent to run a simulation of adaptive sampling. The idea is that we'll just cram our customer data through a harvest[1] loop and call adaptively_sample() on each transaction.

Unfortunately, APM only gives lump-sum per-hour data. That means we won't know when exactly each individual transaction event actually occurs. To get around this problem, I used inverse transform sampling (ITS) to "generate" the finer-grained data that we need. ITS is an algorithm that attempts to model a statistic as the sum of random variables of some type of distribution. In this case, we want a model of the number of transactions that will happen in a given period of time. I included a few different traffic models: one where events occur fairly regularly (i.e. according to a Poisson distribution), one where they vary a little more to represent varying source distance (a squared Poisson distribution), and one representing what I imagine the traffic would look like in a 2-tiered system. (a Compound Poisson distribution). You can find out more about common traffic models [here](https://en.wikipedia.org/wiki/History_of_network_traffic_models#Non-self-similar_traffic_models). Specific details are in the comments of the simulation code below.

[1] Note that when I say "harvest" here, or elsewhere, I just mean "the period of time in which we are trying to gather a target number of sampled=True transactions". Right now, that time period is the exact same time period as the harvest, but they may eventually be different and even out-of-sync.

In [7]:
class BaseHarvester(object):
    def __init__(self, seed, target_samples_per_harvest, harvests_per_hour, **kwargs):
        self.target_samples_per_harvest = target_samples_per_harvest
        self.harvests_per_hour = harvests_per_hour
        self.seconds_per_harvest = 3600/self.harvests_per_hour
        
        self.seed = seed
        self.rand = numpy.random.RandomState(self.seed)
        
        #######################################################
        #######################################################
        ## outputs generated by self.prep_data
        
        # array of ints
        # how many total transactions occur for each harvest
        self.t_per_harvest = []
        
        # array of array of floats
        # one array per harvest
        # each array consists of N+1 floats (for a harvest with N te)
        # each float is the timestep that occurs before the next sample
        self.timestep_per_t_per_harvest = []
        
        # array of floats
        # the absolute time that each harvest occurs at, as HOUR.FRACTIONOFHOUR
        self.harvest_occurence_times = []
        
        # downsampled versions of the data, for faster plot rendering
        self.ds_per_hour = 10
        self.total_num_ds_values = None
        self.harvest_occurence_times_ds = []
        self.t_per_harvest_ds = []
        
        #######################################################
        #######################################################
        ## outputs generated by self.simulate and self.harvest
        
        # an array that will contain the total number of
        # sampled=True te for each harvest in the day
        self.sampledTrue_counts = []
        
        # array of floats, one for each sampled=True te
        # each float is the time relative to the start of the harvest
        # when the te was selected to have sampled=True
        self.sampledTrue_timediffs = []
        
        #######################################################
        #######################################################
        ## outputs generated by self.calc_moving_average
        
        self.ma_range = None
        self.sampledTrue_counts_ma = []

        #######################################################
        #######################################################
        ## used by self.simulate and self.harvest to simulate
        ## the harvest
        
        # the current time (i.e. that would be generated by time.time())
        self.current_timestamp = 0
        
        # the last harvest's te count
        self.last_harvest_t_count = 0
        
        # the current running transaction count so far this harvest 
        self.current_t_count = 0
        
        # the current running sampled=True count so far this harvest
        self.sampledTrue_count = 0
        
        # the time the current harvest starts
        self.current_harvest_start_timestamp = 0
        
    # prep data performs a 2-layered inverse-transform sampling on the 
    # hour-bucketed data from APM
    def prep_data(self, t_per_hour, traffic_model):
        for hour in range(0, len(t_per_hour)):
            for harvest in range(0, self.harvests_per_hour):
                harvest_occurence_time = hour + harvest / float(self.harvests_per_hour)
                self.harvest_occurence_times.append(harvest_occurence_time)
        
        if traffic_model == 'Regular':
            self.generate_poisson_ITS_data(t_per_hour, square_samples=False)
        elif traffic_model == 'Varying':
            self.generate_poisson_ITS_data(t_per_hour, square_samples=True)
        elif traffic_model == 'Compound':
            self.generate_compound_poisson_ITS_data(t_per_hour)
            
        self.downsample_data(t_per_hour)

    # generate an entire hour's worth of transactions, normalize them,
    # then allocate them in order to a harvest
    def generate_poisson_ITS_data(self, t_per_hour, square_samples=False):
        for hour,num_te in enumerate(t_per_hour):
            # generate N exponentially-distributed samples, each having a mean
            # period of num_te / 3600 seconds
            t_frequency = float(num_te) / 3600.0
            unscaled_t_occurences = self.rand.exponential(t_frequency, num_te + 1)
            if square_samples:
                unscaled_t_occurences = unscaled_t_occurences ** 2

            t_occurence_bias = float(numpy.sum(unscaled_t_occurences))
            t_occurence_scaling_factor = 3600.0/t_occurence_bias
            scaled_t_occurences = list(t_occurence_scaling_factor * unscaled_t_occurences)

            # now loop through, keeping a running count. whenever the count excedes the current
            # harvest size, create a new harvest and push the old one onto a list
            current_harvest = []
            current_harvest_progress = 0.0
            current_timestep = scaled_t_occurences.pop(0)
            
            while True:
                # timestep completes the harvest and needs to be split
                if (current_timestep + current_harvest_progress) > self.seconds_per_harvest:
                    self.timestep_per_t_per_harvest.append(current_harvest)
                    self.t_per_harvest.append(len(current_harvest))
                    
                    current_timestep -= (self.seconds_per_harvest - current_harvest_progress)
                    current_harvest = []
                    current_harvest_progress = 0.0

                # timestep slots into current harvest with no fuss
                elif len(scaled_t_occurences) > 0:
                    current_harvest_progress += current_timestep
                    current_harvest.append(current_timestep)
                    current_timestep = scaled_t_occurences.pop(0)

                else:
                    # the last timestep completes the current harvest
                    if current_timestep < 0.001:
                        remnant = self.seconds_per_harvest - sum(self.timestep_per_t_per_harvest[-1])
                        self.timestep_per_t_per_harvest[-1].append(remnant)
                        self.t_per_harvest[-1] += 1

                    # the last timestep is bigger than one harvest, and overflows into the next
                    else:
                        current_harvest_progress += current_timestep
                        current_harvest.append(current_timestep)

                        self.timestep_per_t_per_harvest.append(current_harvest)
                        self.t_per_harvest.append(len(current_harvest))
                        
                    break
                    
    # generate data appropriate for a 2-tier model, such that each tier will have
    # poisson-distributed t_counts
    def generate_compound_poisson_ITS_data(self, t_per_hour):
        for hour in range(0, len(t_per_hour)):
            t_this_hour = self.generate_per_hour_compound_ITS_data(t_per_hour[hour])

            for harvest in range(0, self.harvests_per_hour):
                self.generate_per_harvest_compound_ITS_data(t_this_hour[harvest])

    # first ITS layer
    # assume that within a given hour the number of samples per harvest
    # will be distributed according to a Poisson distribution. Therefore,
    # we can inverse-sample them via the exponential distribution.
    def generate_per_hour_compound_ITS_data(self, t_per_hour):
        # generate H exponentially-distributed samples, each having an average of
        # N transactions, where H = harvets per hour and N = t_per_hour
        t_per_harvest = float(t_per_hour) / float(self.harvests_per_hour)
        unscaled_t_per_hour = self.rand.exponential(t_per_harvest, self.harvests_per_hour)
        
        # the above method doesn't actually generate N te within one hour, but rather samples from an
        # exponential distribution that has a lambda parameter equal to N/harvests samples occuring per
        # hour. thus, there will be some error that we need to factor out.
        t_count_bias = numpy.sum(unscaled_t_per_hour)
        t_count_scaling_factor = t_per_hour / t_count_bias
        scaled_t_per_hour = numpy.rint(t_count_scaling_factor * unscaled_t_per_hour)
        
        # note that scaled_t_per_hour will have a very tiny amount of quantization error
        # i did some testing on the size of it, and IMHO it is not worth correcting for
        self.t_per_harvest = numpy.append(self.t_per_harvest, scaled_t_per_hour)
        return scaled_t_per_hour

    # second ITS layer
    # assume that the number of samples per harvest is *also* a Poisson
    # distribution, meaning that for a given N, we can inversely sample
    # according to the exponential distribution again in order to find an 
    # estimated timestamp for every sample for the entire day.
    def generate_per_harvest_compound_ITS_data(self, t_this_harvest):
        # format x data as "HOUR.FRACTIONOFHOUR" for our plots
        if t_this_harvest == 0:
            self.timestep_per_t_per_harvest.append([])
            return

        # generate N+1 exponentially-distributed samples spread throughout one harvest,
        # where N=t_this_harvest. the +1 will be the delay that occurs before the first
        # element for the first element (and will be discarded during the simulation of
        # the harvest)
        mean_period_between_te = float(self.seconds_per_harvest) / float(t_this_harvest)
        unscaled_t_occurences = self.rand.exponential(
            mean_period_between_te,
            int(t_this_harvest) + 1)

        # as with the per-hour data, the above method doesn't actually generate N+1 te
        # within one harvest, but rather samples from an exponential distribution that
        # has a lambda parameter equal to 1 sample occuring per 1/N seconds. thus,
        # there will be some error that we need to factor out.
        t_occurence_bias = float(numpy.sum(unscaled_t_occurences))
        t_occurence_scaling_factor = float(self.seconds_per_harvest) / t_occurence_bias

        scaled_t_occurences = t_occurence_scaling_factor * unscaled_t_occurences
        self.timestep_per_t_per_harvest.append(scaled_t_occurences)
           
    # downsample our data to just self.ds_per_hour harvests per minute, so our output
    # plots won't get clunky
    def downsample_data(self, t_per_hour):
        self.total_num_ds_values = self.ds_per_hour*len(t_per_hour)
        ds_factor = int(len(self.t_per_harvest)/float(self.total_num_ds_values))
        
        for i in range(0, self.total_num_ds_values):
            self.harvest_occurence_times_ds.append(self.harvest_occurence_times[ds_factor*i])
            self.t_per_harvest_ds.append(self.t_per_harvest[ds_factor*i])

    # requested by Chris Wildman: calculate a smoothed version of the sampled=True count
    # to better gauge adaptive sampling performance
    def calc_moving_average(self, ma_range):
        if self.ma_range == ma_range:
            return

        self.ma_range = ma_range
        
        if hasattr(self, 'original_sampledTrue_counts'):
            stc = self.original_sampledTrue_counts
        else:
            stc = self.sampledTrue_counts
        
        if ma_range == 1:
            sampledTrue_ma = stc

        else:
            sampledTrue_ma = []
            delta = int(ma_range/2)
            
            for i,s in enumerate(stc):
                start_of_range = max(i-delta, 0)
                end_of_range = min(i+delta, len(stc))
                
                length = end_of_range - start_of_range
                samples_to_smooth = stc[start_of_range:end_of_range]
                ma_sample_count = sum(samples_to_smooth)/length
                sampledTrue_ma.append(ma_sample_count)

        # we also want to downsample our ma
        self.sampledTrue_counts_ma = []
        ds_factor = int(len(sampledTrue_ma)/float(self.total_num_ds_values))
        
        for i in range(0, self.total_num_ds_values):
            self.sampledTrue_counts_ma.append(sampledTrue_ma[ds_factor*i])

    #####################################################
    #####################################################
            
    # sum up a histogram range
    def cond_sum(self, a, rl, rh):
        return numpy.sum([numpy.sum(a==n) for n in range(rl, rh)])

    # calculate various output information
    def collate_output(self):
        total_sum = len(self.sampledTrue_counts)
        
        diff_from_middle = self.target_samples_per_harvest // 2
        rl = self.target_samples_per_harvest - diff_from_middle
        rh = self.target_samples_per_harvest + diff_from_middle

        a = numpy.array(self.sampledTrue_counts)
        under_ratio = self.cond_sum(a, 0, rl)
        middle_ratio = self.cond_sum(a, rl, rh)
        over_ratio = self.cond_sum(a, rh, 2 * self.target_samples_per_harvest + 1)

        self.under_percentage = 100.0 * under_ratio / total_sum
        self.middle_percentage = 100.0 * middle_ratio/total_sum
        self.over_percentage = 100.0 * over_ratio / total_sum
        self.outside_percentage = 100.0 - self.middle_percentage
        
        self.sparse_count = sum(numpy.array(self.t_per_harvest) <= rl)
        self.sparse_percentage = 100.0*self.sparse_count / len(self.t_per_harvest)

        self.middle_range = "(%d-%d)" % (rl, rh)
        end_tag = 2*int(self.target_samples_per_harvest)
        
        if diff_from_middle == 1:
            self.sparse_range = "(harvests with exactly 0 transactions)"
            self.under_range = "(0)"
            self.over_range = "(%d)" % end_tag
            self.outside_range = "(0 or %d)" % end_tag
            
        else:
            self.sparse_range = "(harvests with only 0-%d total transactions)" % (rl-1)
            self.under_range = "(0-%d)" % (rl-1)
            self.over_range = "(%d-%d)" % (rh+1, end_tag)
            self.outside_range = "(0-%d, %d-%d)" % (rl-1, rh+1, end_tag)
            
    # put the output info in the output boxes
    def set_output_labels(self, outboxes):
        self.collate_output()
        
        for key in outboxes:
            value = getattr(self, key)
            if not isinstance(value, str):
                value = "%6.2f" % value
            outboxes[key].value = value
            
    # some harvests, especially in the middle of the night, won't actually
    # get enough samples to show the adaptive sampling algorithm does anything
    # so, we can choose to remove them from the histogram and output boxes
    def tidy_sparse_output(self, do_tidying):
        if not hasattr(self, 'original_sampledTrue_counts'):
            self.original_sampledTrue_counts = self.sampledTrue_counts
        
        self.sampledTrue_counts = self.original_sampledTrue_counts
        
        if not do_tidying:
            return

        diff_from_middle = self.target_samples_per_harvest // 2
        rl = self.target_samples_per_harvest - diff_from_middle

        self.original_sampledTrue_counts = self.sampledTrue_counts
        self.sampledTrue_counts = []

        for i in range(0, len(self.t_per_harvest)):
            if self.t_per_harvest[i] > rl:
                self.sampledTrue_counts.append(self.original_sampledTrue_counts[i])

In [8]:
class Harvester(BaseHarvester):
    def simulate(self):
        self.last_harvest_t_count = -1
        
        for i in range(len(self.t_per_harvest)):
            self.index = i
            self.harvest(self.timestep_per_t_per_harvest[i])
            self.last_harvest_t_count = int(self.t_per_harvest[i])
            self.post_harvest_cleanup()
            
    def post_harvest_cleanup(self):
        self.current_harvest_start_timestamp += self.seconds_per_harvest
        self.current_timestamp = self.current_harvest_start_timestamp
        self.current_t_count = 0
            
    # perform a single harvest, counting the number of transactions
    # are assigned sampled=True
    def harvest(self, transaction_events):
        self.harvest_t_count = 0
        self.sampledTrue_count = 0
        
        for i,timestep in enumerate(transaction_events):
            if i == 0:
                # NOTE: this is the time that occurs before the first te of the harvest arrives
                self.current_timestamp += timestep
                continue
                
            self.current_t_count += 1
            timedelta_from_harvest_start = self.current_timestamp - self.current_harvest_start_timestamp
            
            # TODO: truncate microsecond portion of timestep to more accurately simulate time.time()?
            #       or is there a just-as-efficient way to get finer-grained timestamps?
            self.adaptive_sampling_setup(timedelta_from_harvest_start)

            is_sampled = False

            # initial base case, for first harvest
            if self.last_harvest_t_count == -1:
                is_sampled = self.sampledTrue_count < self.target_samples_per_harvest

            if self.sampledTrue_count < 2*self.target_samples_per_harvest:
                priority = self.rand.uniform(0.0, 1.0)
                is_sampled = self.adaptively_sample(priority)
                
            if is_sampled:
                timediff = self.current_timestamp - self.current_harvest_start_timestamp
                self.sampledTrue_timediffs.append(timediff)
                
            self.sampledTrue_count += int(is_sampled)
                
            # NOTE: this is the time that occurs until the *next* te arrives
            self.current_timestamp += timestep

        self.sampledTrue_counts.append(self.sampledTrue_count)

    # if we want to do more stuff to process a transaction event before calling adaptive_sampling,
    # do it here
    def adaptive_sampling_setup(self, timedelta_from_harvest_start):
        pass
    
    # API: returns "True" if we should set Sampled=True, "False" otherwise
    def adaptively_sample(self, priority):
        raise NotImplementedError('IMPLEMENT THIS, YA NUMBSKULL!')

In [9]:
# some orchestration around the harvester to allow us to try out various
# parameters, and then cache the results so that the sliders won't be slow
# due to recalculating the plot every time
current_harvester = None
tas_cache = {}
harvester_classes = {}

def test_adaptive_sampling(class_name, seed, target_samples_per_harvest,
                           harvests_per_hour, ma_range, tidy_output,
                           traffic_model, erlang_window_size,
                           num_burst_protection_factors, rate_estimation_method):
    global harvester_classes
    global current_harvester
    global tas_cache
    
    if len(harvester_classes.keys()) == 0:
        g = globals()
        for name in g:
            if ('Harvester' in name) and (name not in ['BaseHarvester', 'Harvester']):
                bare_name = name.replace('Harvester', '')
                harvester_classes[bare_name] = g[name]
    
    cls = harvester_classes[class_name]
    
    # the fancy bayesian has extra model parameters, so it needs a fancy tag
    if class_name == 'FancyA':
        tag = "%s/%s/%s/%s/%s/%s/%s" % (
            cls.__name__, seed, harvests_per_hour, traffic_model,
            erlang_window_size, num_burst_protection_factors, rate_estimation_method
        )
        
    else:
        tag = "%s/%s/%s/%s" % (cls.__name__, seed, harvests_per_hour, traffic_model)

    # if we've already simulated this harvester, grab it from the cache. otherwise,
    # create it anew and then simulate it
    if tag in tas_cache:
        harvester = tas_cache[tag]
        
    else:
        harvester = cls(seed, target_samples_per_harvest,
                        harvests_per_hour, erlang_window_size=erlang_window_size,
                        num_burst_protection_factors=num_burst_protection_factors,
                        rate_estimation_method=rate_estimation_method)
        
        harvester.prep_data(t_per_hour, traffic_model)
        harvester.simulate()
        tas_cache[tag] = harvester
        
    harvester.tidy_sparse_output(tidy_output)
    harvester.calc_moving_average(ma_range)
    current_harvester = harvester
    return harvester

DHTML(add_toggle_button('simulation', 4, 5, 6))

# Harvesters

I then created some subclasses of the Harvester to implement a few different adaptive sampling algorithms. We'll then evaluate these guys on two different metrics. First, on how precise it gets to our sampling target (10) on various workloads across time. Second, on how unbiased its selection choices are. That is to say, a good algorithm  will choose its events with an equal spread across the whole time period, while a bad algorithm will be biased towards a particular part of the harvest. (most likely, the beginning)

First, we'll do a base case, that just randomly assigns Sampled=True.

In [10]:
# comparison base case: this should perform horribly
class RandomHarvester(Harvester):
    def adaptively_sample(self, priority):
        return priority > 0.5

## Agent Harvesters

Next, we'll add a couple agent implementations: our initial release of the python agent, which didn't have exponential backoff, and the Ruby agent, which does. Once we plot our output, we'll be able to toggle between our two agents to see what sort of added benefit the exponential backoff strategy provides under different workloads.

In [11]:
# to v1 Better-CAT spec, without exponential backoff
# NOTE: after Erika found out we did not implement exponential backoff, she
# thwomped our heads and now the Python Agent has exponential backoff too :)
class PythonAgentHarvester(Harvester):
    def adaptively_sample(self, priority):
        target = self.target_samples_per_harvest
        scaled_target = (target / max(1, self.last_harvest_t_count))
        return priority < scaled_target

In [12]:
# to v1 Better-CAT spec, with exponential backoff
class RubyAgentHarvester(Harvester):
    def adaptively_sample(self, priority):
        target = self.target_samples_per_harvest
        if self.sampledTrue_count > target:
            target = target ** (target / self.sampledTrue_count) - target ** 0.51

        scaled_target = (target / max(1, self.last_harvest_t_count))
        return priority < scaled_target

## Extreme Backoff Harvesters

Next, lets try a few spinoffs on the idea of exponential backoff. These will use TOTALLY MORE EXTREME (!!!) exponential ramping to ensure that we'll get much closer to 10. At first glance, these will perform much better. However! It will also turn out that they have the problem of being highly biased towards the start of the sampling period, and are therefore not too good. If you think about it, they're not much different than just sampling the first 10 transactions in every harvest.

In [13]:
# lower the threshold if we're below the target, raise it once we're past it
class RampBothHarvester(Harvester):
    def adaptively_sample(self, priority):
        target = self.target_samples_per_harvest
        count = 7 if self.sampledTrue_count < 7 else self.sampledTrue_count
        
        main_term = numpy.exp( count ** (target/count) / 5 )
        correction_term = numpy.exp( (20 ** 0.5) / 5 )
        target = main_term - correction_term
        
        scaled_target = (target / max(1, self.last_harvest_t_count))
        return priority < scaled_target

In [14]:
# featuring MORE EXTREME ramping, starting even before we reach our target
class HeavyRampHarvester(Harvester):
    def adaptively_sample(self, priority):
        target = self.target_samples_per_harvest
        count = 4 if self.sampledTrue_count < 4 else self.sampledTrue_count
        
        main_term = numpy.exp ( 2*target / (count-1) - 1 )
        correction_term = 1
        target = main_term - correction_term
        
        scaled_target = (target / max(1, self.last_harvest_t_count))
        return priority < scaled_target

## Adaptive Harvester

Finally, I wanted to try a different type of harvester. It's based on the idea that we can improve our algorithm by adapting our sampling target based on various other pieces of information we have on hand.  I'll try two things:

### Adapting the threshold based on our current progress. 

The problem that this address is trying to compensate a bit for random numbers being, well, random.

Sometimes, even if conditions are otherwise perfect - e.g. our data throughput is perfectly constant from harvest to harvest, our threshold is perfectly set, etc - it is very well possible that we could generate a bunch of priorities that are nowhere near the necessary threshold for sampling. So, what would the effect be if, based on our currently location in the harvest, we slightly adjusted the threshold?

For example, if we're 60% done with the current harvest and yet have only assigned 4 sampled=True events so far, then we can judge that we're under-sampling somewhat, and thus we can temporarily lower the threshold by 33% so that we're more likely to hit our target. On the other hand, if we're 60% done with the harvest and we've assigned 6 out of 10 sampled=True events so far, then we can say we're right on track and therefore don't need to adjust the threshold. 

###  Predicting our expected data rate instead of using the last harvest's count. 

The idea here is that a more accurate number for this harvest's total transaction count will help us gain a more precise number of samples for this harvest.

Using the last harvest's transaction count is not too bad of an estimate near the start of a harvest, it is seriously outdated near the end. At that point, that count includes information about transactions that occurred nearly 2 minutes ago! Furthermore, the last harvest's numbers don't account for the new info that we've accumulated for this current harvest. For example, there might have been a burst of activity in the last harvest, and so its rate is far too high for this harvest, where transactions occur at more usual rate. Or vice versa.

Now, how to predict the rate? Past rate won't necessarily correlate with future rate, so how can we be sure our rate prediction is stable? To answer that question, we can create a linear regression based on [Bayesian Inference](https://en.wikipedia.org/wiki/Bayesian_inference). It can involve a lot of math, but luckily, in this case, it boils down pretty neatly in the end. The gist of it is that we use some sort of prior understanding of the data rate and then continuously update it using new incoming information. We can then use this to generate a prediction that we can be confident will be stable and unbiased. I'll put an explanation of the math in a comment in the code below, if you're curious about the details. The tl;dr is that the running average of the period of time between transaction becomes a better and better prediction on the final transaction count as we get closer and closer to the harvest point.

Thus, we have one estimate that's good near the start of the harvest but bad near the end (the last harvest's transaction count), and another estimate that's bad near the start of the harvest but good near the end. Let's use both by calculating a weighted average between the two based amount of time that's occurred so far this harvest.

### Requirements

To implement both of these adaptations, we'll need, in addition to what's required by Better CAT, the following data:

1. The transaction count of the harvest/sampling-period thus far
2. The time that the current harvest/sampling-period started
3. The time that a given transaction transaction started

We'll then calculate:

1. fhc = the fraction of harvest completed thus far
2. progress_ratio = how high is our sample count relative to how much time has eclipsed in the harvest thus far?
3. rate_estimate = the number of seconds in the harvest divided by the running average of the period between transaction starts

In [15]:
# attempt to predict our current_harvest_count via bayesian inference
class AdaptiveHarvester(Harvester):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # the (F)raction of the (H)arvest (C)ompleted thus far
        self.fhc = 0.0
        
        # we'll update our rate estimate every N transactions, where N=erlang_window_size
        self.erlang_window_size = 10
        
        # our two adaptive factors
        self.rate_estimate = 0
        self.progress_ratio = 1
        
        # unsquared, the average progress_ratio will be roughly equal to
        # 1 - 1/sampling_target,
        # and thus we need to correct on average by 1/(1 - 1/sampling_target)**2
        
        factor = 1.0/(1.0 - 1.0/self.target_samples_per_harvest)        
        self.correction_bias = factor ** 2
        
        # TO CONSIDER: exponential backoff adds extra protection against overshoots,
        #              and so we could achieve better results by biasing our threshold
        #              slightly higher
    
    # use a statistic of the posterior predictive distribution (ppd) to generate
    # a guess at the final transaction count for the current harvest
    # I tried the median and the mean... the median seemed to work slightly 
    # better, but the mean is waaay easier to understand conceptually so
    # lets just go with that
    #
    # see the FancyA harvester for what the Median looks like as well as
    # an explanation behind what the ppd is
    def predict_rate(self, timedelta_from_harvest_start):
        period_estimate = timedelta_from_harvest_start
        if self.current_t_count > 1:
            period_estimate /= float(self.current_t_count-1)
            
        return self.seconds_per_harvest / period_estimate
    
    def adaptive_sampling_setup(self, timedelta_from_harvest_start):
        self.fhc = timedelta_from_harvest_start / self.seconds_per_harvest
        
        # calculate the ratio of expectation / progress, and then square it so
        # that it has more of an influence on the results
        progress = float(self.sampledTrue_count+1) / float(self.target_samples_per_harvest+1)
        expectation = max(self.fhc, 1/self.target_samples_per_harvest)
        self.progress_ratio = (expectation/progress)**2
        
        if (self.current_t_count % self.erlang_window_size) != 0:
            return False
        
        rate_prediction = self.predict_rate(timedelta_from_harvest_start)
    
        # form the rate prediction as a weighted average between last harvest's rate
        # and this harvest's predicted rate, weighted so that once this harvest is 50%
        # complete we'll be going off our predicted_rate only
        fhcf = min(1.0, 2.0*self.fhc)
        self.rate_estimate = (1.0-fhcf)*self.last_harvest_t_count + fhcf*rate_prediction
        return True
    
    def adaptively_sample(self, priority):
        target = self.target_samples_per_harvest
        
        # include exponential backoff
        if self.sampledTrue_count > target:
            target = target ** (target / self.sampledTrue_count ) - target ** 0.51

        target = target * self.progress_ratio * self.correction_bias
        scaled_target = (target / max(1, self.rate_estimate))
        return priority < scaled_target

### Are additional adaptations worthwhile?

Next, we'll try out a few bells and whistles to this adaptive sampler to see if we can improve it any further. For example, I noticed that it has some commonalities with a PID controller, using something akin to the P and I terms (the progress_ratio and the rate_estimate, respectively). What if we add a D term too, will it help? Also, how big can we make that erlang window? Calculating it less often would be nice, but eventually our performance will degrade. Finally, what if we sample from that posterior predictive distribution instead of just using the median or mode? Will that be less biased?

In [16]:
# First, the promised explanation of the bayesian inference method.
#
# The goal is to calculate something called the "posterior predictive
# distribution" (ppd) for the total number of transactions we'll see
# this harvest. Using the ppd makes sure that we're accurately capturing
# the uncertainty about the rate so that we don't end up over-fitting
# our guess based on the data we have seen.
#
# To calcuate the ppd, we first need a distribution for the actual rate
# of transactions-per-harvest based on what we've seen so far during the
# harvest. This is called the "posterior distribution", and we can calculate
# it by integrating the following things:
# 
# 1. A prior model for the transaction rate (number of transactions per harvest), 
#    which says what we might guess what the rate will be with no other information
# 2. A way to calculate what the transaction data rate will likely be based on 
#    observations about it so far.
#
# So, we need to make an assumption on what the distribution of the data will look
# like. If it's a little bit wrong, that's ok - that second term, the likelihood,
# essentially "adjusts" the first term, the prior, based on how well the obvservations
# collected thus far fit it. My choice was to assume it will that the transaction
# period will be distributed according to the exponential distribution. As far as
# I understand it, this is a common pick because it is not only a good fit for
# describing for many kinds of common traffic patterns, but also makes the subsequent
# calculations fairly easy to do, heh. At any rate, the maximum likelihood of the
# exponential distribution is pretty simple, and equal to 1/x, where x is the period
# between samples. Integrating these two yields a posterior distribution for the rate
# based on the "hyperparameters" of the prior distribution.

# However, we don't actually want to know what the distribution for what the
# past transaction rate is. We want to know what the final count will be.
# So, we "marginalize" the posterior distribution over all possibilities of
# the past transaction rate to yield another distribution that is formed from
# only the hyperparameters. That is our ppd. In this case, the math works out
# nicely, and it turns out the ppd is a form of the Pareto distribution, called
# the Lomax distribution, and its two parameters are simple to calculate:
# alpha = number of events so far - 1
# beta = sum of the time of all the events so far, i.e. the difference in time
#        between the start of the harvest and now
# For the most unbiased results, we could generate a random variable from this
# distribution based on these parameters, or just use its mean or median.
#
# Adjusting the rate for every event can yield to a lot of noise, plus is a
# lot of calculation. Another distribution can help us: the Erlang distribution.
# If the period between samples is distributed according to the exponential
# distribution, then the period of N of them is distributed according to the
# Erlang. And, luckily, the maxmimum likelihood of the Erlang distribution is
# just N/x! That means we only need to scale our prediction by the size of the
# window to arrive at the same result.

class FancyAHarvester(AdaptiveHarvester):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.num_burst_protection_factors = kwargs['num_burst_protection_factors']
        self.rate_estimation_method = kwargs['rate_estimation_method']
        self.erlang_window_size = kwargs['erlang_window_size']
        
        self.prev_timesteps = []
        self.last_timestep_timestamp = 0

        bp_factors = range(1, self.num_burst_protection_factors + 1)
        bp_sum = float(sum(bp_factors))
        self.bp_factors = [float(x) / bp_sum for x in bp_factors]
        
        self.max_allowed_timestep_for_bp = float(self.seconds_per_harvest) / 5.0
        self.bp_ratio = 0.10
            
    def post_harvest_cleanup(self):
        leftovers = self.current_t_count % self.erlang_window_size
        super().post_harvest_cleanup()
        self.current_t_count = leftovers
        self.rate_estimate = self.last_harvest_t_count
        
    def lomax(self, lamda, alpha):
        # or random.random or whatever
        x = self.rand.uniform(0, 1, 1)[0]
        
        scale = alpha / lamda
        exponent = -alpha - 1
        base = 1 + x / lamda
        
        return scale * (base**exponent)
    
    def predict_rate(self, timedelta_from_harvest_start):
        if self.current_t_count > 1:
            if 'Lomax Sampled' in self.rate_estimation_method:
                period_estimate = 1.4 * self.lomax(self.current_t_count, timedelta_from_harvest_start)

            elif 'Mean' in self.rate_estimation_method:
                period_estimate = timedelta_from_harvest_start / float(self.current_t_count-1)
                
            elif 'Median' in self.rate_estimation_method:
                exponent = 2.0**(1.0/float(self.current_t_count)) - 1.0
                period_estimate = 1.4 * timedelta_from_harvest_start * exponent
                
            else:
                raise ValueError("Unknown rate estimation method:" + self.rate_estimation_method)
        else:
            period_estimate = timedelta_from_harvest_start
            
        return self.seconds_per_harvest / period_estimate
    
    def adaptive_sampling_setup(self, timedelta_from_harvest_start):
        continuable = super().adaptive_sampling_setup(timedelta_from_harvest_start)
        
        if not continuable:
            return False
        
        if self.num_burst_protection_factors == 0:
            return
        
        current_timestep = self.current_timestamp - self.last_timestep_timestamp
        self.last_timestep_timestamp = self.current_timestamp
        
        # if our t_per_harvest is very low, the short_term estimate won't be helpful
        if (current_timestep > self.max_allowed_timestep_for_bp):
            self.prev_timesteps = []
        else:
            self.prev_timesteps.append(current_timestep)
            
        if len(self.prev_timesteps) < self.num_burst_protection_factors:
            return
            
        short_term_rate_estimate_num = self.erlang_window_size * self.seconds_per_harvest
        short_term_rate_estimate_den = sum([
            self.bp_factors[i] * self.prev_timesteps[i]
                for i in range(0, self.num_burst_protection_factors)])
        
        short_term_rate_estimate = short_term_rate_estimate_num / short_term_rate_estimate_den
        self.prev_timesteps.pop(0)
        
        self.rate_estimate = (1-self.bp_ratio)*self.rate_estimate + self.bp_ratio*short_term_rate_estimate
        
DHTML(add_toggle_button('fancy Adaptive Harvester', 13))

# Plotting

Finally, we'll add some sliders and plot the result. On the left is a view of what our ITS data looks like (although note that it is downsampled somewhat so that the plots appear faster), while on the right is a histogram of transaction_events that were selected to have Sampled=True over the course of the 24 hour period.

Note: you can adjust the sliders to try out different parameters.

In [17]:
initial_data = test_adaptive_sampling(
    initial_harvester, initial_seed, baseline_sph,
    initial_hph, initial_ma_range, tidy_output,
    initial_traffic_model, initial_erlang_window_size,
    initial_num_bpf, initial_rem)

DHTML(add_toggle_button('plotting', 14, 15, 16, 17, 18, 19, 20, 21, 22))

In [18]:
centered = Layout(margin='auto')
margin_auto = Layout(margin='0px 0px 0px 10%')
pad_top = Layout(margin='10px 0px 0px 0px')
margin_auto_pad_top = Layout(margin='10px 0px 0px 10%')
first_column = Layout(width='43%')
second_column = Layout(width='43%')
border_black = Layout(
    border='2px solid black',
    margin='2px 0px 0px 0px',
    padding='2px')

bb_initial_display = 'flex' if initial_harvester == 'FancyA' else 'none'
bb_border_black = Layout(
    border='2px solid black',
    margin='2px 0px 0px 0px',
    padding='2px',
    display=bb_initial_display
)

shrink_figbox = Layout(margin='-20px 0px -20px 0px')

In [19]:
# first set up our scales and axes
bx_sc = LinearScale()
by_sc = LinearScale()
bax_x = Axis(label='Time of Occurence', scale=bx_sc, grid_lines='solid')
bax_y = Axis(label='Transaction Event Count (per harvest)',
             scale=by_sc, side='left', grid_lines='solid',
             orientation='vertical')

hx_sc = LinearScale(min=0, max=2*baseline_sph)
hy_sc = LinearScale()
hax_x = Axis(label='Total sampled=True Count (per harvest)',
             scale=hx_sc, grid_lines='solid')
hax_y = Axis(label='Number of Occurrences', scale=hy_sc,
             side='left', grid_lines='solid', orientation='vertical', )

my_sc = LinearScale(min=0, max=2*baseline_sph)
max_y = Axis(label='Number of Sampled=True', scale=my_sc,
             orientation='vertical', side='left', grid_lines='solid')

tx_sc = LinearScale()
ty_sc = LinearScale()
tax_x = Axis(label='Time Within Harvests When sampled=True Occurs',
             scale=tx_sc, grid_lines='solid')
tax_y = Axis(label='Count', scale=ty_sc, side='left', grid_lines='solid',
             orientation='vertical')

In [20]:
ts = SelectionSlider(
    options=['Regular', 'Varying', 'Compound'],
    value=initial_traffic_model,
    description='Traffic Model:',
)

ss = SelectionSlider(
    options=harvester_classes.keys(),
    value=initial_harvester,
    description='Sampling Algorithm:',
)

rss = SelectionSlider(
    options=[1234567, 23523465, 57433462],
    value=initial_seed,
    description='Random Seed:',
)

hphs = SelectionSlider(
    options=[30, 60, 120, 240],
    value=initial_hph,
    description='Harvests Per Hour:',
    )

mar = SelectionSlider(
    options=[1, 2, 4, 8],
    value=initial_ma_range,
    description='Moving Average Range:',
)

ewss = SelectionSlider(
    options=[5, 10, 20, 40],
    value=initial_erlang_window_size,
    description='Erlang Window Size:',
)

nbpfs = SelectionSlider(
    options=[0, 1, 2, 5, 10],
    value=initial_num_bpf,
    description='Burst Protection Terms:',
)

rems = SelectionSlider(
    options=['Mean', 'Median', 'Lomax Sampled'],
    value=initial_rem,
    description='Rate Estimate:',
)

In [21]:
# input scatter plot
t_scatter = Scatter(
    x=initial_data.harvest_occurence_times_ds,
    y=initial_data.t_per_harvest_ds,
    scales={'x': bx_sc, 'y': by_sc})

# histogram for the count of the number of sampled=True payloads
samples_hist = Hist(
    sample=initial_data.sampledTrue_counts,
    scales={'sample': hx_sc, 'count': hy_sc},
    bins=2*baseline_sph + 1)

moving_average_scatter = Scatter(
    x=initial_data.harvest_occurence_times_ds,
    y=initial_data.sampledTrue_counts_ma,
    scales={'x': bx_sc, 'y': my_sc})

timediff_hist = Hist(
    sample=initial_data.sampledTrue_timediffs,
    scales={'sample': tx_sc, 'count': ty_sc},
    bins=initial_hph)

# red bar in the middle of the samples plot
red_bar_height = 500.0
red_line_hist = Lines(
    x=[baseline_sph, baseline_sph, baseline_sph+1.0, baseline_sph+1.0],
    y=[0, red_bar_height, red_bar_height, 0],
    colors=['red'],
    fill='inside',
    scales={'x': hx_sc, 'y': hy_sc})

red_line_y_base = numpy.zeros(len(initial_data.harvest_occurence_times_ds))
red_line_y = red_line_y_base + baseline_sph
red_line_ma = Lines(
    x=initial_data.harvest_occurence_times_ds,
    y=red_line_y,
    colors=['red'],
    fill='inside',
    scales={'x': hx_sc, 'y': my_sc})

In [22]:
cache_button = Button(
    description='Click to clear the plotting cache.',
    disabled=False,
    button_style='',
    tooltip='Click to clear the plot-cache.',
    layout=pad_top
)

# hookup the cache-buster
def bust_cache(button):
    global tas_cache
    global harvester_classes
    
    tas_cache = {}
    harvester_classes = {}

cache_button.observe(cache_button, names='value')

tidier = ToggleButton(
    value=not(tidy_output),
    description='Display sparse harvests in histogram',
    tooltip='Display sparse harvests in histogram',
    icon=('check-square-o' if not(tidy_output) else 'square-o'),
    layout=pad_top
)

In [23]:
bayesian_buttons = VBox([
    HBox([
        VBox([Label('Bayesian Inference Parameters:'), nbpfs], layout=first_column),
        VBox([ewss, rems])
    ], layout=margin_auto)
], layout=bb_border_black)

output_boxes = {}
output_fields = ['middle', 'under', 'over', 'outside', 'sparse']
for name in output_fields:
    output_boxes[name + '_range'] = Label("")
    output_boxes[name + '_percentage'] = Label("")

ob = output_boxes
output_box = VBox([
    HBox([
        VBox([
            Label('Current Run Statistics:'),
            HBox([Label('percent near target'), ob['middle_range'], Label(':'),
                  ob['middle_percentage'], Label('%')]),
            HBox([Label('percent outside target'), ob['outside_range'], Label(':'),
                  ob['outside_percentage'], Label('%')]),
        ], layout=first_column),
        VBox([
            Label(''),
            HBox([Label('percent over target'), ob['over_range'], Label(':'),
                  ob['over_percentage'], Label('%')]),
            HBox([Label('percent under target'), ob['under_range'], Label(':'),
                  ob['under_percentage'], Label('%')])
        ]),
    ], layout=margin_auto),
    HBox([Label('percent of harvests too sparse to sample'), ob['sparse_range'], Label(':'),
                  ob['sparse_percentage'], Label('%')], layout=margin_auto_pad_top),
], layout=border_black)

In [24]:
def update_plots(slider):
    class_name = ss.value
    random_seed = rss.value
    harvests_per_hour = hphs.value
    moving_average_range = mar.value
    tidy_output = not(tidier.value)
    traffic_model = ts.value
    
    erlang_window_size = ewss.value
    num_burst_protection_factors = nbpfs.value
    rate_estimation_method = rems.value
    
    sph_factor = 60.0/harvests_per_hour
    target_samples_per_harvest = int(baseline_sph*sph_factor)

    tidier.icon = 'check-square-o' if not(tidy_output) else 'square-o'
    bayesian_buttons.layout.display = 'flex' \
        if class_name == 'FancyA' else 'none'
        
    harvester = test_adaptive_sampling(
        class_name, random_seed,
        target_samples_per_harvest, harvests_per_hour,
        moving_average_range, tidy_output, traffic_model,
        erlang_window_size, num_burst_protection_factors,
        rate_estimation_method
    )

    t_scatter.x = harvester.harvest_occurence_times_ds
    t_scatter.y = harvester.t_per_harvest_ds

    hx_sc.max = 2*target_samples_per_harvest
    my_sc.max = 2*target_samples_per_harvest
    max_y.tick_values = range(0, 2*target_samples_per_harvest+1)

    samples_hist.bins = 2*target_samples_per_harvest + 1
    samples_hist.sample = harvester.sampledTrue_counts
    timediff_hist.bins = harvests_per_hour
    timediff_hist.sample = harvester.sampledTrue_timediffs
    moving_average_scatter.y = harvester.sampledTrue_counts_ma

    l = target_samples_per_harvest
    h = target_samples_per_harvest + 1.0
    red_line_hist.x = [l, l, h, h]
    red_line_hist.y = [0, red_bar_height/sph_factor, red_bar_height/sph_factor, 0]

    red_line_y_base = numpy.zeros(len(harvester.harvest_occurence_times_ds))
    red_line_ma.y = red_line_y_base + target_samples_per_harvest

    harvester.set_output_labels(output_boxes)
    
ss.observe(update_plots, names='value')
rss.observe(update_plots, names='value')
hphs.observe(update_plots, names='value')
mar.observe(update_plots, names='value')
tidier.observe(update_plots, names='value')
ts.observe(update_plots, names='value')
ewss.observe(update_plots, names='value')
nbpfs.observe(update_plots, names='value')
rems.observe(update_plots, names='value')

# RESULTS

In [25]:
ds_label = Label(
    value="(for the below plots: the points are downsampled to"
          " 1 out of every 6 for faster plotting speed)")

fig_bar = Figure(
    marks=[t_scatter], axes=[bax_x, bax_y],
    title='Transactions Per Harvest')
fig_diff = Figure(
    marks=[timediff_hist], axes=[tax_x, tax_y],
    title='Histogram of Occurence of Sampled=True')
fig_ma = Figure(
    marks=[moving_average_scatter, red_line_ma], axes=[bax_x, max_y],
    title='Samples Per Harvest (Moving Average)')
fig_hist = Figure(
    marks=[samples_hist, red_line_hist], axes=[hax_x, hax_y],
    title='Histogram of Super-Samples per Harvest')

initial_data.set_output_labels(output_boxes)

display(
    VBox([
        VBox([
            HBox([
                VBox([
                    Label('Model Estimation Parameters:'),
                    mar, hphs, HBox([tidier])
                ], layout=first_column),
                VBox([ts, ss, rss, HBox([cache_button])]),
            ], layout=margin_auto)
        ], layout=border_black),
        bayesian_buttons,
        output_box,
        VBox([
            HBox([fig_hist, fig_diff], layout=shrink_figbox)
        ], layout=border_black),
        VBox([
            HBox([ds_label], layout=margin_auto),
            HBox([fig_bar, fig_ma], layout=shrink_figbox)
        ], layout=border_black),
    ])
)

# Analysis

## Judgement Criteria

As far as I can tell, there's two critera we can use to judge an adaptive sampling algorithm: precision (whether it hits close to its target) and bias (whether sampled=True transactions occur with equal probability at any point in the harvest, versus mostly occuring near the beginning or end).

For the latter, I think eyeballing the final distribution will be sufficient. For the former, we'll say that as long as a sampled=True count is closer to the target than near the end, it will be judged as acceptable. This is, in my opinion, a really wide range... if sampled=True payload counts were like a normal distribution, we should expect around ~95% to be within this middle range.

## Exponential Backoff Comparison

Comparing the original Python Agent implementation vs the Ruby Agent implementation, it seems that exponential backoff helps just a little bit when the transaction rate is regular, but helps quite a lot when the data rate varies more. For example, when harvesting once a minute (i.e. 60 times an hour) and targeting 10 samples per harvest under the "Varying" traffic model, the Python Agent gets near the target 78% of the time, while the Ruby Agent gets there 90% of the time, which is just a 15% improvement. However, switching to the "Compound" traffic model drops the Python Agent down to 35% and the Ruby Agent down to 46% - meaning that exponential backoff now provides a 31% benefit!  

The effect is most prominent when throughput is higher, e.g. near the middle of the day. This is because a percentage-wise difference has a higher absolute difference when it baseline value is higher. At any rate, I think that an exponential back-off strategy, as suggested by the spec, could really help here. I am going to add a few different sampling strategies to compare them. As an initial test, the Ruby Agent's currently implemented strategy improves the "near the target" count from 29% to 40%, which is a huge improvement.

## Adaptive Harvester Results

The adaptive harvester seems to work pretty well! However, the additional bells and whistles don't really affect the final results too much. That's because the acceptance criteria is pretty wide already... that is to say, these extra calculations may help turn 9s and 11s into 10s, but since the base adaptive sampling already gets most harvests into the "acceptable" area there's not many others for the fancy one to help with.

# SCRATCH SPACE

In [26]:
# import scipy.stats
# 
# ch = current_harvester
# ch.te_per_harvest = []
# te_per_hour = 10000
# te_per_harvest = float(te_per_hour) / float(ch.harvests_per_hour)
#
# chx = ch.harvests_per_hour
# unscaled_te_per_hour = ch.rand.exponential(te_per_harvest, chx)
# 
# # finally, we have to round our te_counts back to ints. in doing so,
# # we'll introduce some quantization error, so we'll calculate what
# # that is then add it back 
# discrete_te_per_hour = [int(x) for x in scaled_te_per_hour]
# quant_bias = te_per_hour - sum(discrete_te_per_hour)
# error_step = 1 if quant_bias < 0 else -1
# 
# # figure out which indices got hit the hardest by quantization
# ranked = scipy.stats.rankdata(scaled_te_per_hour_i)
# ranked_i = list(numpy.zeros(chx))
# for x in range(0, chx):
#     ranked_i[int(ranked[x])-1] = x
# 
# if quant_bias > 0:
#     ranked_i.reverse()
# 
# while quant_bias > 0:
#     te_bin = ranked_i.pop(0)
#     discrete_te_per_hour[te_bin] -= error_step
#     quant_bias += error_step

In [27]:
# ch = current_harvester
# te_per_hour = 2000
# num_chunks = int(ch.rand.normal(numpy.sqrt(te_per_hour), numpy.log(te_per_hour)))
# size_of_chunks = ch.rand.poisson(te_per_hour/num_chunks, num_chunks)
# bias_of_chunks = te_per_hour/sum([int(x) for x in size_of_chunks])
# size_of_chunks = [int(x*bias_of_chunks) for x in size_of_chunks]
# sum_of_chunks = sum(size_of_chunks)
# diffs = numpy.zeros(abs(te_per_hour - sum_of_chunks)) + int(te_per_hour > sum_of_chunks)
# for d in diffs:
#     size_of_chunks[int(ch.rand.uniform(0, num_chunks))] += int(d)

# print(ch.rand.exponential(te_per_hour, 20))

In [29]:
numpy.std(current_harvester.sampledTrue_counts)

2.0764558134904019

In [37]:
numpy.histogram(current_harvester.sampledTrue_counts, bins=20, range=(0, 20))



(array([ 93,  42,  42,  39,  48,  35,  48,  58, 100, 174, 235, 321, 152,
         48,   5,   0,   0,   0,   0,   0]),
 array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.]))

In [38]:
numpy.histogram(current_harvester.sampledTrue_counts, bins=20, range=(0, 20))

(array([ 96,  41,  40,  51,  83,  86, 105, 102, 121, 109, 122,  96,  74,
         71,  56,  48,  37,  25,  11,  66]),
 array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.]))